<a href="https://colab.research.google.com/github/DEYIF/IMG-DLBreast/blob/new_resnet/Resnet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Histogram Averaging

In [ ]:
import os
import cv2
import numpy as np

def normalize_image(image):
    # Apply histogram equalization for intensity normalization
    normalized_image = cv2.equalizeHist(image)
    return normalized_image

def normalize_dataset(dataset):
    normalized_dataset = []
    for image in dataset:
        normalized_image = normalize_image(image)
        normalized_dataset.append(normalized_image)
    return normalized_dataset

def load_image(file_path):
    if file_path.endswith('.png'):
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    else:
        print('None')
        image = None
    return image

def load_dataset(directory):
    dataset = []
    filenames = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        image = load_image(file_path)
        if image is None or image.size == 0:
            print(f"Failed to load image: {file_path}")
        else:
            dataset.append(image)
            filenames.append(filename)
    return dataset, filenames

dataset_from_center1, filenames_center1 = load_dataset('/content/drive/MyDrive/Classification/train/mask')


normalized_dataset_center1 = normalize_dataset(dataset_from_center1)


output_directory = '/content/drive/MyDrive/Classification/train/histogram_output'
os.makedirs(output_directory, exist_ok=True)

for image, filename in zip(normalized_dataset_center1, filenames_center1):
    output_path = os.path.join(output_directory, filename)
    cv2.imwrite(output_path, image)

Data Import

In [ ]:
import pandas as pd
import os
import numpy as np
import torch
import matplotlib.image as mpimg
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import cv2
from skimage import io, color
from skimage import color as skcolor
from torchvision import transforms
import torchvision.transforms as T
from PIL import Image
import torchvision.transforms.functional as F

In [ ]:
class DBTData(Dataset):
    def __init__(self, filepath, mode, crop_to=(224,224), resize_to=(256, 256), color=True):
        self._crop_to = crop_to
        self._resize_to = resize_to
        self._color = color
        self.mode = mode
        self.filepath = filepath

        # 根据模式加载对应的 CSV 文件
        if mode == "train":
            df = pd.read_csv("/content/drive/MyDrive/1209full_data/label/train.csv")
            filepath = "/content/drive/MyDrive/1209full_data/train"
        elif mode == "valid":
            df = pd.read_csv("/content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv")
            filepath = "/content/drive/MyDrive/1210_seperate_database/SUSI/validation"
            #SUSI /content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv
            #SUSI /content/drive/MyDrive/1210_seperate_database/SUSI/validation
        elif mode == "test":
            df = pd.read_csv("/content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv")
            filepath = "/content/drive/MyDrive/1210_seperate_database/SUSI/validation"

        # extract label
        y = df["type"].values
        self.len = len(y)
        x = []

        # load images
        for i in range(self.len):
            Slice = df.at[i, "filename"]  # 'filename'
            image_path = os.path.join(filepath, Slice)

            try:
                img = mpimg.imread(image_path)
                if len(img.shape) == 3:  #  RGB image，transform to grey image
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                x.append(img)
            except FileNotFoundError:
                print(f"File {image_path} not found")
                continue

        self.x = x
        self.y = torch.from_numpy(y)

#     def divide_by_255(self, x):
#         return x / 255.0

    def __getitem__(self, idx):
        x = self.x[idx]
        if self._color:
            x = skcolor.gray2rgb(x)  #  RGB image
        y = self.y[idx]
        # CrossEntropyLoss does not expect a one-hot encoded vector, but class indices

        def custom_preprocessing(image):
          # Apply noise to the image
          noisy_image = add_noise_to_image(image)

          # Apply blur to the image
          blurred_image = apply_blur_to_image(noisy_image)

          # Adjust contrast and brightness
          enhanced_image = adjust_contrast_brightness(blurred_image)

          return enhanced_image

        def add_noise_to_image(image):
            # Add noise to the image (customize this function as needed)
            noisy_image = np.clip(image + np.random.normal(loc=0, scale=0.1, size=image.shape), 0, 1)
            return noisy_image

        def apply_blur_to_image(image):
            # Apply blur to the image (customize this function as needed)
            blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
            return blurred_image

        def adjust_contrast_brightness(image_tensor):
            # Convert PyTorch tensor to PIL image for transformations
            pil_img = F.to_pil_image(image_tensor)

            # Adjust contrast (factor=1.5)
            pil_img = F.adjust_contrast(pil_img, contrast_factor=1.5)

            # Adjust brightness (factor=1.2)
            pil_img = F.adjust_brightness(pil_img, brightness_factor=1.2)

            # Convert PIL image back to PyTorch tensor
            enhanced_img_tensor = F.to_tensor(pil_img)

            return enhanced_img_tensor


        x = np.atleast_3d(x)  # I love numpy!
        x = x.astype(np.uint8)
        x = Image.fromarray(x.astype(np.uint8))
#         trans = transforms.Compose([
#             transforms.ToPILImage(),
# #             transforms.CenterCrop(self._crop_to),

#             transforms.Resize(self._resize_to),
#             transforms.ToTensor(),
# #             transforms.Grayscale(num_output_channels=1),
# #             transforms.Normalize(mean=[0.5], std=[0.5])
# #             transforms.Normalize(mean=[0], std=[1/255])
# #             self.divide_by_255
# #             // %255
#         ])
#         x = trans(x)
        # mean_nums = [.275, .275, .275]
        # std_nums = [.197, .197, .197]

        # train_transforms = T.Compose([
        #   T.Resize([299,299]),
        #   T.RandomRotation(degrees=5),
        #   T.RandomHorizontalFlip(),
        #   T.RandomPerspective(),
        #   T.ToTensor(),
        #   T.Normalize(mean_nums, std_nums)])

        # valid_transforms = T.Compose([
        #   T.Resize([299,299]),
        #   T.RandomRotation(degrees=5),
        #   T.RandomHorizontalFlip(),
        #   T.RandomPerspective(),
        #   T.ToTensor(),
        #   T.Normalize(mean_nums, std_nums)
        # ])

        # test_transforms = T.Compose([
        #   T.Resize([299,299]),
        #   T.ToTensor(),
        #   T.Normalize(mean_nums, std_nums)
        # ])


        augmentation_class1 = T.Compose([
          T.RandomRotation(degrees=5),
          T.RandomHorizontalFlip(p=0.5),
          # transforms.RandomResizedCrop(size=(height, width), scale=(0.9, 1.1)),
          T.RandomAffine(degrees=5, translate=(0.1, 0.1))
          # T.ToTensor()
        ])

        datagen = T.Compose([
          T.RandomHorizontalFlip(p=1.0),  # Always apply horizontal flip
          T.RandomVerticalFlip(p=1.0),    # Always apply vertical flip
          T.Pad(padding=10, padding_mode='edge') # Fill missing pixels with the nearest value
          # T.ToTensor()
        ])

        test_trans = T.Compose([
          T.Resize([299,299]),
          T.ToTensor()
        ])

        # Test transform: Ensure consistency with train and valid
        test_transforms = T.Compose([
            T.Resize(self._resize_to),  # Resize to the specified dimensions
            T.ToTensor(),  # Convert to PyTorch tensor
            T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize for consistency
        ])



        if self.mode == "train":
            x = train_transforms(x)
        elif self.mode == "valid":
            x = valid_transforms(x)
        elif self.mode == "test":
            x = test_transforms(x)

        return x, y

    def __len__(self):  # return datalength
        return self.len

# dataset = DBTData('train_phase2')
# train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

test部分修改

In [ ]:
class DBTData(Dataset):
    def __init__(self, filepath, mode, crop_to=(224,224), resize_to=(256, 256), color=True):
        self._crop_to = crop_to
        self._resize_to = resize_to
        self._color = color
        self.mode = mode
        self.filepath = filepath

        # 根据模式加载对应的 CSV 文件
        if mode == "train":
            df = pd.read_csv("/content/drive/MyDrive/1209full_data/label/train.csv")
            filepath = "/content/drive/MyDrive/1209full_data/train"
        elif mode == "valid":
            df = pd.read_csv("/content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv")
            filepath = "/content/drive/MyDrive/1210_seperate_database/SUSI/validation"
            #SUSI /content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv
            #SUSI /content/drive/MyDrive/1210_seperate_database/SUSI/validation
        elif mode == "test":
            df = pd.read_csv("/content/drive/MyDrive/1210_seperate_database/SUSI/validation.csv")
            filepath = "/content/drive/MyDrive/1210_seperate_database/SUSI/validation"

        # extract label
        y = df["type"].values
        self.len = len(y)
        x = []

        # load images
        for i in range(self.len):
            Slice = df.at[i, "filename"]  # 'filename'
            image_path = os.path.join(filepath, Slice)

            try:
                img = mpimg.imread(image_path)
                if len(img.shape) == 3:  #  RGB image，transform to grey image
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                x.append(img)
            except FileNotFoundError:
                print(f"File {image_path} not found")
                continue

        self.x = x
        self.y = torch.from_numpy(y)

#     def divide_by_255(self, x):
#         return x / 255.0

    def __getitem__(self, idx):
        x = self.x[idx]
        if self._color:
            x = skcolor.gray2rgb(x)  #  RGB image
        y = self.y[idx]
        # CrossEntropyLoss does not expect a one-hot encoded vector, but class indices

        def custom_preprocessing(image):
          # Apply noise to the image
          noisy_image = add_noise_to_image(image)

          # Apply blur to the image
          blurred_image = apply_blur_to_image(noisy_image)

          # Adjust contrast and brightness
          enhanced_image = adjust_contrast_brightness(blurred_image)

          return enhanced_image

        def add_noise_to_image(image):
            # Add noise to the image (customize this function as needed)
            noisy_image = np.clip(image + np.random.normal(loc=0, scale=0.1, size=image.shape), 0, 1)
            return noisy_image

        def apply_blur_to_image(image):
            # Apply blur to the image (customize this function as needed)
            blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
            return blurred_image

        def adjust_contrast_brightness(image_tensor):
            # Convert PyTorch tensor to PIL image for transformations
            pil_img = F.to_pil_image(image_tensor)

            # Adjust contrast (factor=1.5)
            pil_img = F.adjust_contrast(pil_img, contrast_factor=1.5)

            # Adjust brightness (factor=1.2)
            pil_img = F.adjust_brightness(pil_img, brightness_factor=1.2)

            # Convert PIL image back to PyTorch tensor
            enhanced_img_tensor = F.to_tensor(pil_img)

            return enhanced_img_tensor


        x = np.atleast_3d(x)  # I love numpy!
        x = x.astype(np.uint8)
        x = Image.fromarray(x.astype(np.uint8))


        augmentation_class1 = T.Compose([
          T.RandomRotation(degrees=5),
          T.RandomHorizontalFlip(p=0.5),
          # transforms.RandomResizedCrop(size=(height, width), scale=(0.9, 1.1)),
          T.RandomAffine(degrees=5, translate=(0.1, 0.1))
          # T.ToTensor()
        ])

        datagen = T.Compose([
          T.RandomHorizontalFlip(p=1.0),  # Always apply horizontal flip
          T.RandomVerticalFlip(p=1.0),    # Always apply vertical flip
          T.Pad(padding=10, padding_mode='edge') # Fill missing pixels with the nearest value
          # T.ToTensor()
        ])

        test_trans = T.Compose([
          T.Resize([299,299]),
          T.ToTensor()
        ])

        test_transforms = T.Compose([
            T.Resize(self._resize_to),  # Resize to the specified dimensions
            T.ToTensor(),  # Convert to PyTorch tensor
        ])

        if self.mode == "train":
            x = test_trans(x)
            x = datagen(x)
            if y == 1:  # Check for class 1
                x = augmentation_class1(x)
        elif self.mode == "valid":
            x = test_trans(x)
        elif self.mode == "test":
            x = test_transforms(x)


        # if self.mode == "train":
        #     x = train_transforms(x)
        # elif self.mode == "valid":
        #     x = valid_transforms(x)
        # elif self.mode == "test":
        #     x = test_transforms(x)

        return x, y

    def __len__(self):  # return datalength
        return self.len

# dataset = DBTData('train_phase2')
# train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

test data path

In [ ]:
# 创建数据集对象，并传入必要的参数
dataset_test = DBTData('train_phase2', 'test', crop_to=(224, 224), resize_to=(256, 256), color=True)

# 打印数据集的样本数
print(f"测试样本数量：{len(dataset_test)}")

# 获取第一个样本并打印标签
sample_x, sample_y = dataset_test[0]
print(f"第一个样本的标签：{sample_y}")

测试样本数量：125
第一个样本的标签：0


Train

In [ ]:
# %%

import matplotlib
matplotlib.use('Agg')  # headless plotting
import matplotlib.pyplot as plt
import torch
# from models import BaselineResNet, BayesianResNet
import torchvision.models as models
from torch.utils.data import DataLoader,random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import argparse
import tqdm
import numpy as np
from google.colab import drive
import sys

#add path
sys.path.append('/content/drive/MyDrive/1209run')

#import package above
from dataImport import DBTData
from utils import accuracy

!pip install tensorboardX
from tensorboardX import SummaryWriter
import os

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
# enable cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# setup tensorboardx
writer = SummaryWriter()

# dimension properties
batch_size = 10
val_batch_size = batch_size
num_workers = 0
num_classes = 2
lambda_prop = 0.01

# load data
# load data
color = True
resize_to = (256, 256)
dataset_train = DBTData('train_phase2','train',crop_to=(224,224), resize_to=resize_to, color=color)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataset_valid = DBTData('train_phase2','valid',crop_to=(512,512), resize_to=resize_to, color=color)
dataloader_valid = DataLoader(dataset_valid, batch_size=val_batch_size, num_workers=num_workers)

assert len(dataset_train) > 0
assert len(dataset_valid) > 0
print("Train dataset length:", len(dataset_train))
print("Valid dataset length:", len(dataset_valid))
print('')

# create a model
model = torch.nn.Module()
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)
model = model.to(device)

# create your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4,betas=(0.9, 0.999),weight_decay=1e-8)
lr_scheduler = ReduceLROnPlateau(optimizer, patience=5)

# create loss function
criterion = torch.nn.CrossEntropyLoss()

print('')  # print empty line before training output

# save accuracies and losses during training
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

start_epoch = 0
epochs = 100
e = 0
batch_counter = 0
batch_counter_valid = 0

for e in range(start_epoch, epochs):

    # go through training set
    model.train()
    print("lr =", optimizer.param_groups[0]['lr'])

    epoch_train_loss = []
    epoch_train_acc = []
    is_best = False

    x, y, y_pred = None, None, None
    batches = tqdm.tqdm(dataloader_train)
    for x, y in batches:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        y_pred = model(x)
        train_loss = criterion(y_pred, y)

        train_loss.backward()
        optimizer.step()

        # print current loss
        batches.set_description("loss: {:4f}".format(train_loss.item()))

        # sum epoch loss
        epoch_train_loss.append(train_loss.item())

        # calculate batch train accuracy
        batch_acc = accuracy(y_pred, y)
        epoch_train_acc.append(batch_acc)

        writer.add_scalar('data/train_loss', train_loss.item(), batch_counter)
        writer.add_scalar('data/train_acc', batch_acc, batch_counter)
        batch_counter += 1

    epoch_train_loss = np.mean(epoch_train_loss)
    epoch_train_acc = np.mean(epoch_train_acc)
    lr_scheduler.step(epoch_train_loss)

    # go through validation set
    model.eval()
    with torch.no_grad():

        epoch_valid_loss = []
        epoch_valid_acc = []

        batches = tqdm.tqdm(dataloader_valid)
        for x, y in batches:
            x, y = x.to(device), y.to(device)

            y_pred = model(x)
            valid_loss = criterion(y_pred, y)

            # print current loss
            batches.set_description("loss: {:4f}".format(valid_loss.item()))

            # sum epoch loss
            epoch_valid_loss.append(valid_loss.item())

            # calculate batch train accuracy
            batch_acc = accuracy(y_pred, y)
            epoch_valid_acc.append(batch_acc)

            writer.add_scalar('data/valid_classifier_loss', valid_loss.item(), batch_counter_valid)
            writer.add_scalar('data/valid_acc', batch_acc, batch_counter_valid)
            batch_counter_valid += 1

    epoch_valid_loss = np.mean(epoch_valid_loss)
    epoch_valid_acc = np.mean(epoch_valid_acc)

    print("Epoch {:d}: loss: {:4f}, acc: {:4f}, val_loss: {:4f}, val_acc: {:4f}"
          .format(e,
                  epoch_train_loss,
                  epoch_train_acc,
                  epoch_valid_loss,
                  epoch_valid_acc,
                  ))

    # save epoch losses
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_acc)
    valid_losses.append(epoch_valid_loss)
    valid_accuracies.append(epoch_valid_acc)

    if valid_losses[-1] <= np.min(valid_losses):
        is_best = True

    if is_best:
        filename = "/content/drive/MyDrive/1210_seperate_database/SUSI/snapshots" + "_best.pth.tar"
        print("Saving best weights so far with val_loss: {:4f}".format(valid_losses[-1]))
        torch.save({
            'epoch': e,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'train_losses': train_losses,
            'train_accs': train_accuracies,
            'val_losses': valid_losses,
            'val_accs': valid_accuracies,
        }, filename)

    if e == epochs-1:
      #define Google Drive path folder
        import os
        from google.colab import drive

        #Google Drive
        drive.mount('/content/drive')

        #define Google Drive path folder
        filename = "/content/drive/MyDrive/1210_seperate_database/SUSI/trainmodels" + "_" + str(e) + ".pth.tar"
        print("Saving weights at epoch {:d}".format(e))
        torch.save({
            'epoch': e,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'train_losses': train_losses,
            'train_accs': train_accuracies,
            'val_losses': valid_losses,
            'val_accs': valid_accuracies,
            }, filename)

    print('')

    # plot losses
    plt.figure()
    plt.plot(range(len(train_losses)), train_losses, marker='x')
    plt.plot(range(len(valid_losses)), valid_losses, marker='x')
    plt.title("loss")
    plt.xlabel("epoch")
    plt.ylabel("loss")

    plt.savefig("_loss.pdf", dpi=300)
    plt.figure()
    plt.plot(range(len(train_accuracies)), train_accuracies, marker='x')
    plt.plot(range(len(valid_accuracies)), valid_accuracies, marker='x')
    plt.title("accuracy")
    plt.xlabel("epoch")
    plt.ylabel("acc")
    plt.savefig("acc.pdf", dpi=300)
    plt.close('all')

Train dataset length: 2492
Valid dataset length: 278



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



lr = 0.0001


loss: 0.713948:  76%|███████▌  | 190/250 [00:58<00:18,  3.22it/s]


KeyboardInterrupt: 

Test

In [ ]:
#!/usr/bin/env python3

# Max-Heinrich Laves
# Institute of Mechatronic Systems
# Leibniz Universität Hannover, Germany
# 2019

import torch
import numpy as np
from skimage import io
from sklearn import metrics

torch.manual_seed(0)
np.random.seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from torch.utils.data import DataLoader
import torchvision.models as models
import argparse
import tqdm

#add path
sys.path.append('/content/drive/MyDrive/1209run')

from dataImport import DBTData
from utils import accuracy
from sklearn.metrics import confusion_matrix
import tensorflow as tf

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.metrics import classification_report, f1_score, recall_score
from torchvision.models import shufflenet_v2_x1_0
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import tqdm
import numpy as np
import seaborn as sns

In [ ]:
# 创建数据集对象，并传入必要的参数
dataset_test = DBTData('train_phase2', 'test', crop_to=(224, 224), resize_to=(256, 256), color=True)

# 打印数据集的样本数
print(f"测试样本数量：{len(dataset_test)}")

# 获取第一个样本并打印标签
sample_x, sample_y = dataset_test[0]
print(f"第一个样本的标签：{sample_y}")

测试样本数量：278
第一个样本的标签：0


In [ ]:
dataloader_test = DataLoader(dataset_test, batch_size=val_batch_size, shuffle=False)

model.eval()
with torch.no_grad():
    for i, (x, y) in enumerate(dataloader_test):
        print(f"Batch {i}: x={type(x)}, y={type(y)}")  # Debugging
        x, y = x.to(device), y.to(device)
        y_pred = model(x)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
# enable cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# properties
batch_size = 10
val_batch_size = 1
num_classes = 2
num_mc = 50

# load data
color = True
resize_to = (256, 256)
color = True
resize_to = (256, 256)
snapshot="/content/drive/MyDrive/1209run/snapshots_best.pth.tar"
dataset_test = DBTData('train_phase2',"valid",crop_to=(224,224), resize_to=resize_to, color=color)
dataloader_test = DataLoader(dataset_test, batch_size=val_batch_size, shuffle=False)

assert len(dataset_test) > 0

print("Test dataset length:", len(dataset_test))
print('')

# create a model
model = torch.nn.Module()
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)
model = model.to(device)

# load weights for flow estimation from best last stage
checkpoint = torch.load(snapshot, map_location=device)
print("Loading previous weights at epoch " + str(checkpoint['epoch']) + " from " + snapshot)
model.load_state_dict(checkpoint['state_dict'])

# Create loss function
criterion = torch.nn.CrossEntropyLoss()

# Go through test set
print("Going through test set.")
model.eval()
y_true_np = []
y_pred_np = []
y_pred_prob_np = []

with torch.no_grad():
    test_losses = []
    test_accuracies = []
    correct_var = []
    incorrect_var = []

    batches = tqdm.tqdm(dataloader_test)
    for i, (x, y) in enumerate(batches):
        x, y = x.to(device), y.to(device)

        y_pred = model(x)

        # Use softmax to get the predicted probabilities
        y_pred_prob = F.softmax(y_pred, dim=1)

        y_true_np.append(y.data.cpu().numpy())
        y_pred_np.append(y_pred_prob.argmax(dim=1).data.cpu().numpy())
        y_pred_prob_np.append(y_pred_prob.data.cpu().numpy())

        y = y.to(torch.long)
        mean = y_pred_prob.to(torch.float32)

        test_loss = criterion(mean, y)
        test_losses.append(test_loss.item())

print("test mean loss:", np.mean(test_losses))

y_pred_np = np.array(y_pred_np).squeeze()
y_true_np = np.array(y_true_np).squeeze()
y_pred_prob_np = np.array(y_pred_prob_np).squeeze()

# Save the prediction probabilities
np.save("mc_output_" + str(num_mc) + "_probabilities" + ".npy", y_pred_prob_np)

# Calculate weighted F1 score
f1 = f1_score(y_true_np, y_pred_np, average='weighted')
print("Weighted F1 score:", f1)

# Calculate weighted recall
recall = recall_score(y_true_np, y_pred_np, average='weighted')
print("Weighted recall:", recall)

# Compute AUC for each class
unique_classes = np.unique(y_true_np)

if len(unique_classes) > 1:
    try:
        # Compute AUC score
        auc_score = roc_auc_score(y_true_np, y_pred_prob_np[:, 1], average='weighted')
        print("AUC:", auc_score)

        # --- Add ROC Curve Calculation and Plotting ---
        fpr, tpr, thresholds = roc_curve(y_true_np, y_pred_prob_np[:, 1])  # Compute ROC values
        roc_auc = auc(fpr, tpr)  # Compute the area under the curve (AUC)

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label='Random Guess')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('BUSBRA_Receiver Operating Characteristic (ROC)')
        plt.legend(loc="lower right")
        plt.savefig("BUSBRA_ROC_Curve.pdf", dpi=300)  # Save the plot
        plt.show()
        print("ROC curve plotted and saved.")
    except ValueError as e:
        print(f"Error in computing AUC or plotting ROC curve: {e}")
else:
    print(f"AUC cannot be computed because y_true contains only one class: {unique_classes}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_np, y_pred_np)
print(conf_matrix)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.title("Confusion Matrix")
plt.savefig("Confusion Matrix.pdf", dpi=300)

Test dataset length: 278



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-6-eab702e42e14>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detail

Loading previous weights at epoch 15 from /content/drive/MyDrive/1209run/snapshots_best.pth.tar
Going through test set.


100%|██████████| 278/278 [00:04<00:00, 58.46it/s]


test mean loss: 0.5440681664634952
Weighted F1 score: 0.7652524701916448
Weighted recall: 0.7661870503597122
AUC: 0.8179324440619621
ROC curve plotted and saved.
[[136  30]
 [ 35  77]]
